## Étape 1 : Importer les bibliothèques nécessaires

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MultiLabelBinarizer
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import mlflow
import mlflow.sklearn

## Étape 2 : Charger le fichier CSV

In [2]:
# Charger le DataFrame depuis le fichier CSV nettoyé
df = pd.read_csv('stackoverflow_questions_cleaned.csv')

# Vérifier les données importées
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10046 entries, 0 to 10045
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   date              10046 non-null  object
 1   title             10046 non-null  object
 2   tags              10046 non-null  object
 3   score             10046 non-null  int64 
 4   answer_count      10046 non-null  int64 
 5   sentence_bow      10046 non-null  object
 6   sentence_bow_lem  10046 non-null  object
 7   sentence_dl       10046 non-null  object
dtypes: int64(2), object(6)
memory usage: 628.0+ KB
None


,date,title,tags,score,answer_count,sentence_bow,sentence_bow_lem,sentence_dl
0,2024-07-16 21:13:48,Is it possible to use to use ssim to find diff...,"['python', 'histogram', 'ssim']",1,1,possible use use ssim find differences two ima...,possible use use ssim find difference two imag...,is it possible to use to use ssim to find diff...
1,2024-07-16 20:54:40,Python: Converting string to integer,"['python', 'function', 'integer', 'string-conv...",1,2,python converting string integer,python converting string integer,python converting string to integer
2,2024-07-16 20:44:16,How can I get commands in Rails Migration file...,"['ruby-on-rails', 'regex', 'ruby', 'migration']",1,1,how get commands rails migration file using regex,how get command rail migration file using regex,how can i get commands in rails migration file...
3,2024-07-16 20:43:53,Why do you exclude negative numbers from a max...,['algorithm'],2,1,why exclude negative numbers max path sum algo...,why exclude negative number max path sum algor...,why do you exclude negative numbers from a max...
4,2024-07-16 20:37:24,How can I filter an ISODate field based only t...,"['mongodb', 'mongodb-query', 'aggregation-fram...",2,1,how filter isodate field based time mongodb,how filter isodate field based time mongodb,how can i filter an isodate field based only t...


## Étape 3 : Limiter les tags aux 30 plus fréquents et les encoder

In [3]:
# Créer une liste de tous les tags
all_tags = [tag for tags in df['tags'].apply(eval) for tag in tags]  # Utiliser eval pour convertir les chaînes de listes en listes

# Limiter les tags aux 30 plus fréquents
top_30_tags = [tag for tag, count in Counter(all_tags).most_common(30)]

# Filtrer les tags pour ne garder que les top 30
df['filtered_tags'] = df['tags'].apply(lambda tags: [tag for tag in eval(tags) if tag in top_30_tags])

# Supprimer les lignes sans tags
df = df[df['filtered_tags'].map(len) > 0]

# Encoder les tags avec MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=top_30_tags)
y = mlb.fit_transform(df['filtered_tags'])

# Afficher les tags encodés
print("Classes:", mlb.classes_)
print("Tags encodés:\n", y[:5])

Classes: ['python' 'javascript' 'c#' 'r' 'c++' 'angular' 'java' 'typescript'
 'reactjs' 'html' 'c' 'css' 'android' 'pandas' 'sql' 'excel' 'php'
 'dataframe' 'kotlin' 'flutter' 'postgresql' 'swift' 'node.js'
 'powershell' '.net' 'django' 'ios' 'android-jetpack-compose' 'go' 'numpy']
Tags encodés:
 [[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


## Étape 4 : Transformer les titres en vecteurs TF-IDF

In [19]:
# Utiliser TfidfVectorizer pour encoder les titres
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf_vectorizer.fit_transform(df['sentence_bow'])

# Afficher la forme de la matrice TF-IDF
print("Shape de la matrice TF-IDF:", X_tfidf.shape)

Shape de la matrice TF-IDF: (7109, 5000)


## Étape 5 : Réduire la dimensionnalité des vecteurs TF-IDF avec TruncatedSVD

In [20]:
# Réduire la dimensionnalité des vecteurs TF-IDF
svd = TruncatedSVD(n_components=1500)
X_reduced = svd.fit_transform(X_tfidf)

# Afficher la forme de la matrice réduite
print("Shape de la matrice réduite:", X_reduced.shape)

Shape de la matrice réduite: (7109, 1500)


In [21]:
var_explained = svd.explained_variance_ratio_.sum()
var_explained

0.7943866910027216

## Étape 6 : Entraîner un modèle de classification multilabel

### Modèle avec TF-IDF sans réduction dimensionnelle

In [7]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Démarrer une nouvelle run MLflow avec un nom explicite
mlflow.start_run(run_name="TF-IDF Logistic Regression")

# Entraîner le modèle OneVsRestClassifier avec LogisticRegression
model = OneVsRestClassifier(LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)

# Prédire sur l'ensemble de test
y_pred = model.predict(X_test)

# Calculer et afficher le score F1
f1 = f1_score(y_test, y_pred, average='micro')
print("Score F1 (micro):", f1)

# Loguer les paramètres
mlflow.log_param("max_features", 5000)
mlflow.log_param("test_size", 0.2)
mlflow.log_param("model", "LogisticRegression - TF-IDF")

# Loguer la métrique
mlflow.log_metric("f1_score_micro", f1)

# Loguer le modèle sans wrapper
mlflow.sklearn.log_model(model, "model")

# Loguer la transformation TF-IDF pour la reproductibilité
mlflow.sklearn.log_model(tfidf_vectorizer, "tfidf_vectorizer")

mlflow.end_run()

print("Modèle TF-IDF logué dans MLflow.")

Score F1 (micro): 0.2719703977798335


2024/07/16 22:02:14 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


Modèle TF-IDF logué dans MLflow.


### Modèle avec TF-IDF avec réduction dimensionnelle (TruncatedSVD)

In [8]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.2, random_state=42)

# Démarrer une nouvelle run MLflow avec un nom explicite
mlflow.start_run(run_name="TF-IDF + SVD Logistic Regression")

# Entraîner le modèle OneVsRestClassifier avec LogisticRegression
model = OneVsRestClassifier(LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)

# Prédire sur l'ensemble de test
y_pred = model.predict(X_test)

# Calculer et afficher le score F1
f1 = f1_score(y_test, y_pred, average='micro')
print("Score F1 (micro):", f1)

# Loguer les paramètres
mlflow.log_param("max_features", 5000)
mlflow.log_param("n_components", 300)
mlflow.log_param("test_size", 0.2)
mlflow.log_param("model", "LogisticRegression - TF-IDF + SVD")

# Loguer la métrique
mlflow.log_metric("f1_score_micro", f1)

# Loguer le modèle sans wrapper
mlflow.sklearn.log_model(model, "model")

# Loguer la transformation TF-IDF et SVD pour la reproductibilité
mlflow.sklearn.log_model(tfidf_vectorizer, "tfidf_vectorizer")
mlflow.sklearn.log_model(svd, "svd")

mlflow.end_run()

print("Modèle TF-IDF + SVD logué dans MLflow.")

Score F1 (micro): 0.27821280515891295


2024/07/16 22:02:19 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2024/07/16 22:02:20 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


Modèle TF-IDF + SVD logué dans MLflow.


## Prédire des tags sur une nouvelle question

In [11]:
# Remplacer par l'ID de run de votre modèle logué
run_id = "ec51b67c8da343c58bcdf425f2b4359a"

# Charger le modèle
model = mlflow.sklearn.load_model(f"runs:/{run_id}/model")

# Charger le transformateur TF-IDF
tfidf_vectorizer = mlflow.sklearn.load_model(f"runs:/{run_id}/tfidf_vectorizer")

# Charger le transformateur SVD (si applicable)
try:
    svd = mlflow.sklearn.load_model(f"runs:/{run_id}/svd")
except:
    svd = None

# Exemple de nouvelle question
new_question = "How to use pandas to read a csv file?"

# Transformer la nouvelle question avec TF-IDF
X_tfidf_new = tfidf_vectorizer.transform([new_question])

# Réduire la dimensionnalité avec SVD si applicable
if svd:
    X_new = svd.transform(X_tfidf_new)
else:
    X_new = X_tfidf_new

# Prédire les tags pour la nouvelle question
predicted_tags = model.predict(X_new)

# Utiliser les mêmes tags que ceux utilisés lors de l'entraînement
top_30_tags = [tag for tag, count in Counter(all_tags).most_common(30)]
mlb = MultiLabelBinarizer(classes=top_30_tags)
mlb.fit([top_30_tags])

# Interpréter les résultats
predicted_tag_names = mlb.inverse_transform(predicted_tags)

print(f"Tags prédits pour la nouvelle question: {predicted_tag_names}")

Tags prédits pour la nouvelle question: [('python', 'c#')]
